In [ ]:
from qiskit_nature.drivers import PySCFDriver
from qiskit_nature.transformers import FreezeCoreTransformer

molecule = 'Li 0.0 0.0 0.0; H 0.0 0.0 1.5474'
driver = PySCFDriver(atom=molecule)
qmolecule = driver.run()

# by trial and error, we find that these two orbitals have the highest energy and can be stripped
# core electrons are frozen by default
fctransformer=FreezeCoreTransformer(remove_orbitals=[3,4])
transformer_list=[fctransformer]

print("original info: ")
# print(dir(qmolecule))
print(qmolecule.num_alpha,qmolecule.num_beta,qmolecule.num_molecular_orbitals)
qmolecule=fctransformer.transform(qmolecule)
print(qmolecule.num_alpha,qmolecule.num_beta,qmolecule.num_molecular_orbitals)

In [ ]:
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
problem = ElectronicStructureProblem(driver,q_molecule_transformers=transformer_list)

# Generate the second-quantized operators
second_q_ops = problem.second_q_ops()

# Hamiltonian
main_op = second_q_ops[0]

In [ ]:
from qiskit_nature.mappers.second_quantization import ParityMapper, BravyiKitaevMapper, JordanWignerMapper
from qiskit_nature.converters.second_quantization.qubit_converter import QubitConverter

# Setup the mapper and qubit converter
# parity mapper supports two qubit reduction which can help in reduction of cost of the circuit
mapper_type = 'ParityMapper'

if mapper_type == 'ParityMapper':
    mapper = ParityMapper()
elif mapper_type == 'JordanWignerMapper':
    mapper = JordanWignerMapper()
elif mapper_type == 'BravyiKitaevMapper':
    mapper = BravyiKitaevMapper()

# z2 symmetry is not present after these modifications as can be seen in the cell after the next one
converter = QubitConverter(mapper=mapper, two_qubit_reduction=True,z2symmetry_reduction=None)

# The fermionic operators are mapped to qubit operators
num_particles = (problem.molecule_data_transformed.num_alpha,
             problem.molecule_data_transformed.num_beta)
qubit_op = converter.convert(main_op, num_particles=num_particles)

In [ ]:
print(qubit_op)

In [ ]:
from qiskit.opflow.primitive_ops import Z2Symmetries


print(Z2Symmetries.find_Z2_symmetries(qubit_op))

In [ ]:
from qiskit_nature.circuit.library import HartreeFock

num_particles = (problem.molecule_data_transformed.num_alpha,
             problem.molecule_data_transformed.num_beta)
num_spin_orbitals = 2 * problem.molecule_data_transformed.num_molecular_orbitals
init_state = HartreeFock(num_spin_orbitals, num_particles, converter)
print(init_state)

In [ ]:
from qiskit.circuit.library import TwoLocal
from qiskit_nature.circuit.library import UCCSD, PUCCD, SUCCD

# Choose the ansatz
# I observed that ucc ansatz converge fast and have very few parameters
# They also converge to the required value with just one rep
# however they have a high cost. 
# So first start with a high cost twolocal then move to custom once twolocal cant be further optimised
ansatz_type = "Custom"

# Parameters for q-UCC antatze
num_particles = (problem.molecule_data_transformed.num_alpha,
             problem.molecule_data_transformed.num_beta)
num_spin_orbitals = 2 * problem.molecule_data_transformed.num_molecular_orbitals

# Put arguments for twolocal
if ansatz_type == "TwoLocal":
    # Single qubit rotations that are placed on all qubits with independent parameters
    rotation_blocks = ['ry', 'rz']
    # Entangling gates
    entanglement_blocks = 'cx'
    # How the qubits are entangled 
    entanglement = 'circular'
    # Repetitions of rotation_blocks + entanglement_blocks with independent parameters
    repetitions = 4
    # Skip the final rotation_blocks layer
    skip_final_rotation_layer = True
    ansatz = TwoLocal(qubit_op.num_qubits, rotation_blocks, entanglement_blocks, reps=repetitions, 
                      entanglement=entanglement, skip_final_rotation_layer=skip_final_rotation_layer)
    # Add the initial state
    ansatz.compose(init_state, front=True, inplace=True)
elif ansatz_type == "UCCSD":
    ansatz = UCCSD(converter,num_particles,num_spin_orbitals,initial_state = init_state)
elif ansatz_type == "PUCCD":
    ansatz = PUCCD(converter,num_particles,num_spin_orbitals,initial_state = init_state)
elif ansatz_type == "SUCCD":
    ansatz = SUCCD(converter,num_particles,num_spin_orbitals,initial_state = init_state)
elif ansatz_type == "Custom":
    # this is essentially same as a twolocal ansatz 
    # with the only difference being the single qubit gates after the last cnots
    
    from qiskit.circuit import Parameter, QuantumCircuit, QuantumRegister, ParameterVector
    # Define the variational parameter
    no_reps=1
    single_rot_rep=1
    n = qubit_op.num_qubits
    theta = ParameterVector('a',length=n*no_reps*3*single_rot_rep+3*n*single_rot_rep)
    # Make an empty quantum circuit
    qc = QuantumCircuit(qubit_op.num_qubits)
    used_params=0
    for i in range(no_reps):
        for k in range(single_rot_rep):
            for j in range(n):
                # instead of ry and rz as in twolocal 
                # use rz,ry,rz since every single qubit unitary can be written as a product of these three gates
                qc.rz(theta[used_params],j)
                used_params+=1
                qc.ry(theta[used_params],j)
                used_params+=1
                qc.rz(theta[used_params],j)
                used_params+=1
        start_qubit=3
#         for j in range(start_qubit,n+start_qubit-1):
#             if j%n==1:
#                 continue
#             qc.cx(j%n,(j+1)%n)
#         qc.cx(3,4)
#         qc.cx(4,5)
#         qc.cx(5,2)
        qc.cx(0,1)
        qc.cx(1,2)
        qc.cx(2,3)
#         qc.cx(3,0)
#         qc.cx(0,1)
        
    for k in range(single_rot_rep):
        for j in range(n):
                qc.rz(theta[used_params],j)
                used_params+=1
                qc.ry(theta[used_params],j)
                used_params+=1
                qc.rz(theta[used_params],j)
                used_params+=1
    ansatz = qc
    ansatz.compose(init_state, front=True, inplace=True)

print(ansatz)

In [ ]:
from qiskit import Aer
backend = Aer.get_backend('statevector_simulator')

In [ ]:
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B, SPSA, SLSQP, AQGD, P_BFGS

optimizer_type = 'COBYLA'

# try each optimizer as all have different algorithms to search the resulting space 
# for low cost circuits, any optimizer would work well
if optimizer_type == 'COBYLA':
    optimizer = COBYLA(maxiter=20000)
elif optimizer_type == 'L_BFGS_B':
    optimizer = L_BFGS_B(maxfun=10000)
elif optimizer_type == 'SPSA':
    optimizer = SPSA(maxiter=20000)
elif optimizer_type == 'SLSQP':
    optimizer = SLSQP(maxiter=10000)
elif optimizer_type=='AQGD':
    optimizer=AQGD(maxiter=1000)
elif optimizer_type=='P_BFGS':
    optimizer=P_BFGS(maxfun=10000)

In [ ]:
from qiskit_nature.algorithms.ground_state_solvers.minimum_eigensolver_factories import NumPyMinimumEigensolverFactory
from qiskit_nature.algorithms.ground_state_solvers import GroundStateEigensolver
import numpy as np 

def exact_diagonalizer(problem, converter):
    solver = NumPyMinimumEigensolverFactory()
    calc = GroundStateEigensolver(converter, solver)
    result = calc.solve(problem)
    return result

result_exact = exact_diagonalizer(problem, converter)
exact_energy = np.real(result_exact.eigenenergies[0])
print("Exact electronic energy", exact_energy)
print(result_exact)

# Check with your VQE result.

In [ ]:
from qiskit.algorithms import VQE
from IPython.display import display, clear_output

# Print and save the data in lists
def callback(eval_count, parameters, mean, std):  
    # Overwrites the same line when printing
    display("Evaluation: {}, Energy: {}, Std: {}".format(eval_count, mean, std))
    clear_output(wait=True)
    counts.append(eval_count)
    values.append(mean)
    params.append(parameters)
    deviation.append(std)

counts = []
values = []
params = []
deviation = []

# Set initial parameters of the ansatz
# We choose a fixed small displacement 
# So all participants start from similar starting point
try:
    initial_point = [0.01] * len(ansatz.ordered_parameters)
except:
    initial_point = [0.01] * ansatz.num_parameters

algorithm = VQE(ansatz,
                optimizer=optimizer,
                quantum_instance=backend,
                callback=callback,
                initial_point=initial_point)

result = algorithm.compute_minimum_eigenvalue(qubit_op)

print(result)

In [ ]:
# Store results in a dictionary
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import Unroller

# Unroller transpile your circuit into CNOTs and U gates
pass_ = Unroller(['u', 'cx'])
pm = PassManager(pass_)
ansatz_tp = pm.run(ansatz)
cnots = ansatz_tp.count_ops()['cx']
score = cnots

accuracy_threshold = 4.0 # in mHa
energy = result.optimal_value

if ansatz_type == "TwoLocal":
    result_dict = {
        'optimizer': optimizer.__class__.__name__,
        'mapping': converter.mapper.__class__.__name__,
        'ansatz': ansatz.__class__.__name__,
        'rotation blocks': rotation_blocks,
        'entanglement_blocks': entanglement_blocks,
        'entanglement': entanglement,
        'repetitions': repetitions,
        'skip_final_rotation_layer': skip_final_rotation_layer,
        'energy (Ha)': energy,
        'error (mHa)': (energy-exact_energy)*1000,
        'pass': (energy-exact_energy)*1000 <= accuracy_threshold,
        '# of parameters': len(result.optimal_point),
        'final parameters': result.optimal_point,
        '# of evaluations': result.optimizer_evals,
        'optimizer time': result.optimizer_time,
        '# of qubits': int(qubit_op.num_qubits),
        '# of CNOTs': cnots,
        'score': score}
else:
    result_dict = {
        'optimizer': optimizer.__class__.__name__,
        'mapping': converter.mapper.__class__.__name__,
        'ansatz': ansatz.__class__.__name__,
        'rotation blocks': None,
        'entanglement_blocks': None,
        'entanglement': None,
        'repetitions': None,
        'skip_final_rotation_layer': None,
        'energy (Ha)': energy,
        'error (mHa)': (energy-exact_energy)*1000,
        'pass': (energy-exact_energy)*1000 <= accuracy_threshold,
        '# of parameters': len(result.optimal_point),
        'final parameters': result.optimal_point,
        '# of evaluations': result.optimizer_evals,
        'optimizer time': result.optimizer_time,
        '# of qubits': int(qubit_op.num_qubits),
        '# of CNOTs': cnots,
        'score': score}

# Plot the results
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 1)
ax.set_xlabel('Iterations')
ax.set_ylabel('Energy')
ax.grid()
fig.text(0.7, 0.75, f'Energy: {result.optimal_value:.3f}\nScore: {score:.0f}')
plt.title(f"{result_dict['optimizer']}-{result_dict['mapping']}\n{result_dict['ansatz']}")
ax.plot(counts, values)
ax.axhline(exact_energy, linestyle='--')
fig_title = f"\
{result_dict['optimizer']}-\
{result_dict['mapping']}-\
{result_dict['ansatz']}-\
Energy({result_dict['energy (Ha)']:.3f})-\
Score({result_dict['score']:.0f})\
.png"
fig.savefig(fig_title, dpi=300)

# Display and save the data
import pandas as pd
import os.path
filename = 'results_h2.csv'
if os.path.isfile(filename):
    result_df = pd.read_csv(filename)
    result_df = result_df.append([result_dict])
else:
    result_df = pd.DataFrame.from_dict([result_dict])
result_df.to_csv(filename)
result_df[['optimizer','ansatz', '# of qubits', '# of parameters','rotation blocks', 'entanglement_blocks',
    'entanglement', 'repetitions', 'error (mHa)', 'pass', 'score']]